In [48]:
!nvidia-smi

Tue Mar 21 21:05:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    34W / 250W |   1587MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [49]:
import os
from time import time
from tqdm import tqdm
import numpy
import pandas as pd

import torch
import torch
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

import torchvision
from torchvision.datasets import ImageFolder
from torchvision.models import resnet18
from torchvision.transforms import transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import tifffile
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

torch version: 1.13.0
torchvision version: 0.14.0


In [50]:
import torchvision.models as models
import torch
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True)
model.eval()

# define VGG16 model
mobilenet_v2 = models.mobilenet_v2(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()
print(use_cuda)
# move model to GPU if CUDA is available
if use_cuda:
    mobilenet_v2 = mobilenet_v2.cuda()
# define VGG16 model
mobilenet_v2 = models.mobilenet_v2(pretrained=True)

# check if CUDA is available
use_cuda = torch.cuda.is_available()
print(use_cuda)
# move model to GPU if CUDA is available
if use_cuda:
    mobilenet_v2 = mobilenet_v2.cuda()

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


True
True


In [63]:
import os
from torchvision import datasets
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import time
from PIL import Image
import torchvision.transforms as transforms
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

from torch.utils.data import random_split

### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes


# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 32


data_dir = '/kaggle/input/messidor1-data/P_Data'
train_dir = os.path.join(data_dir, 'Train/')
test_dir = os.path.join(data_dir, 'Test/')

# percentage of training set to use as validation
valid_size = 0.2


transform = { 'train'}

data_transforms = {
    'train' : transforms.Compose([transforms.Resize(128),
                                  transforms.CenterCrop(128),
                                  transforms.RandomHorizontalFlip(), # randomly flip and rotate
                                  transforms.RandomRotation(10),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.3288, 0.1636, 0.0559],
                                  std=[0.3691, 0.1875, 0.0723])]),

    
    'test' : transforms.Compose([transforms.Resize(128),
                                  transforms.CenterCrop(128),
                                  transforms.ToTensor(),
                                  transforms.Normalize(mean=[0.3288, 0.1636, 0.0559],
                                  std=[0.3691, 0.1875, 0.0723])])}

image_datasets = {'train' : datasets.ImageFolder(root=train_dir,transform=data_transforms['train']),
                  
                  'test' : datasets.ImageFolder(root=test_dir,transform=data_transforms['test']),}

# split the training set into training and validation sets
train_dataset, valid_dataset = random_split(image_datasets['train'], 
                                            [int((1 - valid_size) * len(image_datasets['train'])), 
                                             int(valid_size * len(image_datasets['train']))])
# define the data loaders for the training, validation, and test sets
loaders_transfer = {
    'train': torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True),
    'valid': torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=batch_size, num_workers=num_workers, shuffle=True)
}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train',  'test']}
class_names = image_datasets['train'].classes
n_classes = len(class_names)

print( len(class_names))
print( len(image_datasets['train']))

print( len(image_datasets['test']))
print( len(loaders_transfer['valid']))

4
800
400
5


In [52]:
print(len(mobilenet_v2.classifier))

2


In [53]:
print(mobilenet_v2)
print(mobilenet_v2.classifier[1].in_features) 
print(mobilenet_v2.classifier[1].out_features)

MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [54]:
import torchvision.models as models
import torch.nn as nn


## TODO: Specify model architecture 

# Load the pretrained model from pytorch
model_transfer = models.mobilenet_v2(pretrained=True)

feats_list = list(model_transfer.features)
new_feats_list = []
for feat in feats_list:
    new_feats_list.append(feat)
    if isinstance(feat, nn.Conv2d):
        new_feats_list.append(nn.Dropout(p=0.5, inplace=True))

# modify convolution layers
model_transfer.features = nn.Sequential(*new_feats_list)


for param in model_transfer.features.parameters():
    param.requires_grad = False


n_inputs = model_transfer.classifier[1].in_features

last_layer = nn.Linear(n_inputs, 4)

model_transfer.classifier[1] = last_layer
if use_cuda:
    model_transfer = model_transfer.cuda()

In [55]:
import torch.optim as optim

criterion_transfer = nn.CrossEntropyLoss()
optimizer_transfer = optim.Adam(model_transfer.classifier.parameters(), lr=0.001)

In [56]:
train_loss_list =[]
train_acc_list = []
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    total_time = 0
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss  = 0.0
        correct_train = 0.
        total_train = 0.
        
        valid_loss = 0.0

        ###################
        # train the model #
        ###################
        model.train()
        start = timeit.default_timer()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1] #miraj
            # _, predicted = torch.max(output.data, 1)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss += loss.item()
            total_train += target.size(0)
            correct_train += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())#(predicted == target).sum().item()
            
          
            data.size(0)
            ###
        stop = timeit.default_timer() 
        new_time = stop-start
        total_time+=new_time
        ######################
        # validate the model #
        ######################
        model.eval()

        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            # update the average validation loss

            ##
            #forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            preds = torch.nn.functional.log_softmax(output , dim=1)

            #calculate the batch loss
            loss = criterion(output, target)
            #update average validation loss 
            valid_loss += loss.item()
            data.size(0)
            ##
        train_loss = train_loss/len(loaders['train'].dataset) ###
        train_loss_list.append(train_loss)
        train_acc_list.append(correct_train / total_train)
        valid_loss = valid_loss/len(loaders['valid'].dataset) ###


        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f}\tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss

    # return trained model
    print("total training time: ", total_time)
    return model

In [57]:
# train the model
import timeit
import numpy as np
model_transfer = train(20, loaders_transfer, model_transfer, optimizer_transfer, criterion_transfer, use_cuda, '/kaggle/working/model_transfer.pt')


Epoch: 1 	Training Loss: 0.039434	Validation Loss: 0.037903
Validation loss decreased (inf --> 0.037903).  Saving model ...
Epoch: 2 	Training Loss: 0.035484	Validation Loss: 0.035702
Validation loss decreased (0.037903 --> 0.035702).  Saving model ...
Epoch: 3 	Training Loss: 0.033455	Validation Loss: 0.035418
Validation loss decreased (0.035702 --> 0.035418).  Saving model ...
Epoch: 4 	Training Loss: 0.033370	Validation Loss: 0.036095
Epoch: 5 	Training Loss: 0.031487	Validation Loss: 0.036072
Epoch: 6 	Training Loss: 0.030673	Validation Loss: 0.036058
Epoch: 7 	Training Loss: 0.031685	Validation Loss: 0.037441
Epoch: 8 	Training Loss: 0.030703	Validation Loss: 0.037075
Epoch: 9 	Training Loss: 0.029536	Validation Loss: 0.036708
Epoch: 10 	Training Loss: 0.029867	Validation Loss: 0.037121
Epoch: 11 	Training Loss: 0.028830	Validation Loss: 0.035839
Epoch: 12 	Training Loss: 0.028186	Validation Loss: 0.036611
Epoch: 13 	Training Loss: 0.028716	Validation Loss: 0.035902
Epoch: 14 	Tra

In [58]:
model_transfer.load_state_dict(torch.load('/kaggle/working/model_transfer.pt'))

<All keys matched successfully>

In [59]:
test_loss_list = []
test_acc_list = []
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
   
    correct = 0.0
    total = 0.0
    t_time = 0
    model.eval()
    start = timeit.default_timer()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        test_loss_list.append(test_loss)
        total += data.size(0)
        test_acc_list.append(correct / total)
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
        
    stop = timeit.default_timer()
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    print()
    print("total time ", stop-start)

In [64]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)


Test Accuracy: 53% (17/32)

Test Accuracy: 57% (37/64)

Test Accuracy: 59% (57/96)

Test Accuracy: 57% (73/128)

Test Accuracy: 57% (92/160)

Test Accuracy: 55% (107/192)

Test Accuracy: 53% (119/224)

Test Accuracy: 54% (139/256)

Test Accuracy: 52% (152/288)

Test Accuracy: 51% (166/320)

Test Accuracy: 50% (178/352)

Test Accuracy: 50% (192/384)

Test Accuracy: 50% (200/400)
Test Loss: 1.298897


Test Accuracy: 50% (200/400)

total time  12.267449592000048
